In [1]:
import math

millnames = ['',' tys',' mln',' mld',' tln']
def humanReadable_large(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))
    return '{:.0f}{}'.format(n / 10**(3 * millidx), millnames[millidx])

In [2]:
from random import randint
import time

dataLength = 10000000;

start = time.time();

x = [randint(1, 5) for p in range(dataLength)];

end = time.time();

print('Czas losowania danych: ' + str(round(end-start, 4)) + 's, ilość danych: ' + humanReadable_large(dataLength));

Czas losowania danych: 10.6942s, ilość danych: 10 mln


### Metoda 1: Klasyczne `sum() / len()`:

In [3]:
start = time.time();

avg = sum(x) / len(x);

end = time.time();

print('Czas: ' + str(round(end-start, 4)) + 's, wartość: ' + str(avg));

Czas: 0.0768s, wartość: 3.0006273


### Metoda 2: Klasyczna średnia:

In [4]:
start = time.time();

amount = 0;
for item in x:
    amount += item;
avg = amount / len(x);

end = time.time();

print('Czas: ' + str(round(end-start, 4)) + 's, wartość: ' + str(avg));

Czas: 0.7706s, wartość: 3.0006273


### Metoda 3: MapReduce (bez podziału na części + bez wątków):

In [5]:
arr = [];

start = time.time();

for i in set(x):
    arr.append([i, x.count(i)]);

full_sum = 0;
for (i, j) in arr:
    full_sum += (i * j);

avg = full_sum / len(x);

end = time.time();

print('Czas: ' + str(round(end-start, 4)) + 's, wartość: ' + str(avg));

Czas: 0.7861s, wartość: 3.0006273


### Metoda 4: MapReduce (podział na części + wątki):

In [6]:
def chunkenizer(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [7]:
def map_reduce(lst, results):
    for i in set(lst):   # Mapuj
        if i in results: # Redukuj
            results[i] = results[i] + lst.count(i);
        else:
            results[i] = lst.count(i);

Zawsze: `[value, counter]`

In [8]:
import threading
import numpy as np

chunks = 1000000;

start = time.time();

splitted = list(chunkenizer(x, chunks));

results = {};
threads = [];
for i in splitted:
    thread = threading.Thread(target=map_reduce, args=(i, results,));
    threads.append(thread);
    thread.start();
    thread.join();

full_sum = 0;
for key, value in results.items():
    full_sum += int(key) * value;

avg = full_sum / len(x);

end = time.time();

print('Czas: ' + str(round(end-start, 4)) + 's, wartość: ' + str(avg) + ', ilość kawalków: ' + humanReadable_large(chunks));

Czas: 0.8197s, wartość: 3.0006273, ilość kawalków: 1 mln


### Metoda 5: Wykorzystanie `statistics.mean()`:

In [9]:
import statistics

start = time.time();

avg = statistics.mean(x);

end = time.time();

print('Czas: ' + str(round(end-start, 4)) + 's, wartość: ' + str(avg));

Czas: 5.4856s, wartość: 3.0006273
